<h1 align=center><font size=6><b>Random Forests</b></font></h1>
<hr>

## **Summary**
#### **Decision trees**

The basis of tree-based learners is the decision tree wherein a series of decision rules (e.g., “If their gender is male...”) are chained. The result looks vaguely like an upside-down tree, with the first decision rule at the top and subsequent decision rules spreading out below. In a decision tree, every decision rule occurs at a decision node, with the rule creating branches leading to new nodes. A branch without a decision rule at the end is called a *leaf*. <font color=red><b>Note: the interpretability of decision tree is great.</b></font>

`Sklearn.DecisionTreeClassifier` use Gini impurity:
$$G(t) = 1 - \sum_{i = 1}^{c}p_{i}^{2}$$
where $G(t)$ is the Gini impurity at node $t$ and $p_i$ is the proportion of observations of the class $c$ at node $t$.

<b>Decision tree regressor</b>, instead of reducing Gini impurity or entropy, potential splits are by default measured on how much reduce the MSE:
$$\text{MSE} = \frac{1}{n}\sum_{i = 1}^{n}(y_i - \hat{y}_i)^2$$

#### **Random Forest ––– an ensamble method of the decision trees**

* <font size=3, color=brown>Aggregating the predictions of a group of predictors(e.g., classifiers or regressors) $\Longrightarrow$ better predictions than with the best individual predictor</font>
* <font size=3>Using the Ensemble methods after attaining a few good predictors towards to the end of a project</font>
* <font size=3>e.g., Netflix Prize Competition</font>
* <font size=3>Ensemble methods: _bagging, boosting, stacking, etc._</font>

<h2><font size = 4.5>Table of Content</font></h2>

<hr>

<div class="alert alert-block alert-info" style="margin-top: 10px">
<li><a href="#ref1">1. Voting Classifier</a></li>
<ul><div><li><a href="#ref1">1.1 Training and Visualizing</a></div>
<div><li><a href="#ref2">1.2 Making Predictions</a></div>
<div><li><a href="#ref3">1.3 Model Intepretation</a></div>
<div><li><a href="#ref4">1.4 Estimating Class Probablities</a></div></ul>
<li><a href="#ref5">2. The CART Training Algorithm</a></li>
<li><a href="#ref6">3. Gini Impurity vs Entropy</a></li>
<li><a href="#ref7">4. Regularization Hyperparameters</a></li>
<li><a href="#ref8">5. Regression</a></li>
<ul><div><li><a href="#ref9">5.1 Classification and Regression Tree (CART) algothrim tries to minimize the MSE</a></li></div></ul>
<li><a href="#ref10">6. Instability</a></li>
<li><a href="#ref11">7. Interactive Visualization</a></li>
<li><a href="#ref12">8. Interesting References</a></li>
    
<hr>

### **1. Voting Classifiers**
#### **1.1 Definition**
* <font size=3, color=brown>Having multiple classifier: e.g., Logistic Regression, SVM, Random Forest, k-Nearest Neighbors, and other classifiers</font>
<img src="fig7_1.png" width=500>

* <font size=3>Majority-vote classifier (<font color=red>getting the most votes</font>) $\Longrightarrow$ ***Hard Voting*** classifier</font>
* <font size=3>If all classifiers can estimate class probabilities, SciKit-Learn  can predict the class with the highest class probability, averaged over all the individual classifier$\Longrightarrow$ ***Soft Voting*** classifier</font>
<img src="fig7_2.png" width=500>

* <font size=3, color=green>The more independent predictors are, the better the ensemle methods work $\Longrightarrow$ Using different algorithms</font>

#### **1.2 Example using the moons dataset**

In [1]:
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
df_moons = make_moons()
X = df_moons[0]
y = df_moons[1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state =58)

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf_hard = SVC()
svm_clf_soft = SVC(probability=True)

# Hard voting
voting_clf_hard = VotingClassifier(estimators=[('lr', log_clf), 
                                               ('rf', rnd_clf), 
                                               ('svc', svm_clf_hard)],
                                   voting = 'hard')

voting_clf_hard.fit(X_train, y_train)
# soft voting
voting_clf_soft = VotingClassifier(estimators=[('lr', log_clf), 
                                               ('rf', rnd_clf), 
                                               ('svc', svm_clf_soft)],
                                   voting = 'soft')

voting_clf_soft.fit(X_train, y_train)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                             

In [3]:
from sklearn.metrics import accuracy_score

print("Results of Hard Voting:")
for clf in (log_clf, rnd_clf, svm_clf_hard, voting_clf_hard):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
    
print("Results of Soft Voting:")
for clf in (log_clf, rnd_clf, svm_clf_soft, voting_clf_soft):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

Results of Hard Voting:
LogisticRegression 0.8
RandomForestClassifier 0.85
SVC 1.0
VotingClassifier 0.85
Results of Soft Voting:
LogisticRegression 0.8
RandomForestClassifier 0.85
SVC 1.0
VotingClassifier 0.85


### **2. Bagging and Pasting** 
* <font size=3><b>Bootstrapping</b> the training data w/ and w/o replacement, respectively</font>
* <font size=3>Train the model with random subsets of the training set with the same training algorithm</font>

In [4]:
# decision boundary plot function
def plot_decision_boundary(clf, X, Y, cmap='Paired_r'):
    h = 0.02
    x_min, x_max = X[:,0].min() - 10*h, X[:,0].max() + 10*h
    y_min, y_max = X[:,1].min() - 10*h, X[:,1].max() + 10*h
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.figure(figsize=(8,5))
    plt.contourf(xx, yy, Z, cmap=cmap, alpha=0.25)
    plt.contour(xx, yy, Z, colors='k', linewidths=0.7)
    plt.scatter(X[:,0], X[:,1], c=Y, cmap=cmap, edgecolors='k');

#### **2.1 Example**
* <font size=3>Training 500 classifiers with 100 instances randomly sampled from the training set with replacement</font>
* <font size=3>n_jobs: the number of CPU cores to use (-1 indicate all cores)</font>

In [5]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(DecisionTreeClassifier(), 
                            n_estimators = 500, 
                            max_samples = 80, 
                            bootstrap=True, 
                            n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred =  bag_clf.predict(X_test)

#### **2.2 Out-of-Bag Evaluation**
* <font size=3>BaggingClassifier samples _m_ training instances with replacement, where _m_ is the size of the training set</font>
* <font size=3>k = _n- m_ (n: all instances): not sampled in the training model $\Longrightarrow$ ***out-of-bag*** (oob) instances</font>
* <font size=3>k instances can be used for cross-validation

In [6]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators = 500, max_samples = 80, bootstrap=True, n_jobs=-1, oob_score = True)
bag_clf.fit(X_train, y_train)
print('Out-of-Bag score:', bag_clf.oob_score_)

from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
print('Accuracy socre:', accuracy_score(y_test, y_pred))

print('Out-of-Bag Decision Function:', bag_clf.oob_decision_function_[:])

Out-of-Bag score: 0.95
Accuracy socre: 1.0
Out-of-Bag Decision Function: [[0.         1.        ]
 [0.00540541 0.99459459]
 [1.         0.        ]
 [0.91326531 0.08673469]
 [0.06508876 0.93491124]
 [0.         1.        ]
 [0.05945946 0.94054054]
 [0.9950495  0.0049505 ]
 [0.86813187 0.13186813]
 [0.         1.        ]
 [1.         0.        ]
 [0.         1.        ]
 [0.45294118 0.54705882]
 [0.0960452  0.9039548 ]
 [0.16939891 0.83060109]
 [1.         0.        ]
 [0.08823529 0.91176471]
 [0.1        0.9       ]
 [0.13714286 0.86285714]
 [0.87978142 0.12021858]
 [0.95604396 0.04395604]
 [0.         1.        ]
 [0.05617978 0.94382022]
 [0.22099448 0.77900552]
 [0.48913043 0.51086957]
 [0.9516129  0.0483871 ]
 [0.77245509 0.22754491]
 [0.87830688 0.12169312]
 [1.         0.        ]
 [0.         1.        ]
 [0.92195122 0.07804878]
 [0.05660377 0.94339623]
 [1.         0.        ]
 [1.         0.        ]
 [0.70621469 0.29378531]
 [0.05699482 0.94300518]
 [0.08571429 0.91428571]
 [

### **3. Random Patches and Random Subspaces**
* <font size=3>BaggingClassifier: can sampling features as well</font>
* <font size=3>Sampling both instances and features $\Longrightarrow$ <font color=brown>_Random Patches_ method</font> (max_features, bootstrap_features)</font>
* <font size=3>Sampling only features $\Longrightarrow$ <font color=orange>_Random Subspaces_ method</font> (boostrap=False, max_samples=1.0, bootstrap_features =True, max_features < 1.0)</font>

### **4. Random Forests**
* <font size=3>An ensemble of Decision Trees, generally trained via bagging method (sometimes pasting method)</font>
* <font size=3 color=brown>_RandomForestClassifier_$\Longrightarrow$ more convenient and optimized for Decision Trees</font>
* <font size=3 color=brown>_RandomForestRegressor_</font>
* <font size=3>_RandomForstClassifier_: ALL the hyperparameters of <font color=orange>_DecisionTreeClassifier_</font> + <font color=orange>_BaggingClassifier_</font></font>
* <font size=3> Searching for the BEST feature AMONG random subsets of features, instead of the very best feature when splitting a node</font>

In [7]:
from sklearn.ensemble import RandomForestClassifier


rnd_clf = RandomForestClassifier(n_estimators = 500, max_leaf_nodes = 16, n_jobs = -1)
rnd_clf.fit(X_train, y_train)

y_pred = rnd_clf.predict(X_test)

# equal to bagging method as below
bag_clf = BaggingClassifier(DecisionTreeClassifier(splitter='random', max_leaf_nodes=16),
                           n_estimators = 500, max_samples = 1.0, bootstrap = True, n_jobs = -1)

#### **4.1 Extremely Randomized Trees (Extra-Trees)**
* <font size=3>Applying random thresholds for each features rather than searching for the best possible thresholds</font>
* <font size=3 color=brown>Trading more bias for a lower variance</font>
* <font size=3>Much faster than regular Random Forests</font>

#### **4.2 Feature Importance**
* <font size=3>More important features: close to the root; less important features: close to leaves</font>
* <font size=3>Feature importance: computing the average depth across all trees in the forest</font>

In [8]:
# Iris dataset
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators = 500, n_jobs = -1)
rnd_clf.fit(iris["data"], iris['target'])
for name, score in zip(iris['feature_names'], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.08627603608936044
sepal width (cm) 0.02230614110986166
petal length (cm) 0.4376025458314549
petal width (cm) 0.45381527696932306


In [9]:
# MINST dataset example

# find the data location of the Scikit-Learn Data
from sklearn.datasets.base import get_data_home 
print (get_data_home())

from sklearn.datasets import fetch_mldata
#mnist = fetch_mldata('MNIST original')
#X, y = mnist['data'], mnist['target']

#rnd_clf.fit(X, y)
#feature_score = rnd_clf.feature_importances_


/Users/mengchen/scikit_learn_data


### **5. Boosting**
* <font size=3><font color=orange>Any Ensemble method</font> combining several weak learners into a strong learner $\Longrightarrow$ Training each classifier <font color=brown>sequentially</font>; each subsequent classifier tries to correct the predecessor</font>
* <font size=3>_AdaBoost_(_Adaptive Boosting_) + _Gradient Boosting_</font>

#### **5.1 AdoBoosting**
* <font size=3>Focusing more and more on the hard cases</font>
* <font size=3>Increasing the relative weight of misclassified training instances $\Longrightarrow$ sequentially training the model with updated weights, so on</font>
<img src="fig7_7.png" width=500>

* <font size=3.5, color=brown>AdaBoost Algorithm</font>

  * <font size=3>Initially, $w^{(i)} = \frac{1}{m}$, weighted error rate of the $j^{th}$ predictor</font> 
    $$r_{j} = \frac{\displaystyle\sum_{\substack{i=1 \\
                               \hat{y}_{j}^{(i)}\neq{y^{(i)}}
                               }}^{m}w^{(i)}}
                               {\displaystyle\sum_{i=1}^{m}w^{(i)}}$$
    <font size=3>where $\hat{y}_{j}^{(i)}$ is the $j^{th}$ predictor's prediction fro the $i^{th}$ instance.</font>
  * <font size=3>Predictor weight
    $$\alpha_{j} = \eta log\frac{1-r_j}{r_j}$$
    <font size=3>the $\alpha\uparrow$, the more accurate the predictor is.</font>
  * <font size=3>Boosting the misclassified instances with updated weights</font>
    for $i = 1,2,..., m$
    $$w^{(i)}\leftarrow
        \begin{cases}
            w^{(i)}                 & \text{if }\hat{y}_{j}^{(i)} = y^{(i)} \\
            w^{(i)}exp\big(\alpha_{j}\big)  & \text{if }\hat{y}_{j}^{(i)}\neq{y^{(i)}}
        \end{cases}
    $$
    <font size=3>then all the instance weights are normalized (i.e., divided by $\sum_{i=1}^{m}w^{(i)}$).</font>
  * <font size=3>Stops until the desired number of predictors is reached, or a perfect predictor is found</font>
  * <font size=3>AdaBoost Prediction: computing the predictors and weighs them using the predictor weights $\alpha_{j}$.
    $$\hat{y}(x) = \argmax_c
                   \displaystyle\sum_{\substack{j=1 \\
                               \hat{y}_{j}(x)= k
                               }}^{N}\alpha_{j}
    $$
    <font size=3>where $N$ is the number of predictors<font>
  * <font size=3 color=brown>Scikit-Learn use a multiclass version of AdaBoost called _SAMME_ (Stagwise Additive Modeling using a Multiclass Exponential loss function)</font>
            _SAMME.R_ (R stands for 'Real') for estimating probabilities

In [10]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth = 1),  # max_depth --> Decision Stump is a Decision Tree with max_depth = 1: A tree composed of 
                             n_estimators = 200,                     # a single decision node plus two leaf nodes
                             algorithm = 'SAMME.R',                  # If AdaBoost ensemble is overfitting, try reducing the number of estimators or more strongly
                             learning_rate = 0.5)                    # regularizing the base estimator
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=0.5, n_estimators=200, random_state=None)

#### **5.2 Gradient Boosting**
* <font size=3>Fitting the new predictor to the <font color=red>_residual errors_</font> made by the predeccessor</font>

In [11]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X, y)

y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X, y2)

y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X, y3)

y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

# ALTERNATIVE Approach
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth =  2, n_estimators = 3, learning_rate = 1.0)
gbrt.fit(X, y)

NameError: name 'X_new' is not defined

<img src='fig7_9.png' width = 500>

#### <font color=orange>***5.2.1 Shrinkage***</font>: setting the learning_rate hyperparameter low to generate more trees for better predictions
<img src='fig7_10.png' width=500>

#### <font color=orange>***5.2.2 Optimal Number of Trees***</font>
* <font size=3>using <font color=brown>*staged_predict()*</font> method, measuring the validation error at each stage of training to find the optimal number of trees, and finally train another GBRT ensemble using the optimal number of trees</font>

In [13]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

errors = [mean_squred_error(y_val, y_pred)
          for y_pred in gbrt.predict(X_val)]
bst_n_estimators =  np.argmin(errors)

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

NameError: name 'GradientBoostingRegressor' is not defined

* <font size=3>Early Stopping Implementation: setting <font color=green>*warm_start</font>=True*, keeping exsiting trees when the <font color=green>_fit()_</font> method is called</font>
* <font size=3 color=brown>Stochastic Gradient Boosting</font>
    * <font size=3>hyperparameter <font color=green>*subsample*</font>: specifies the fraction of training instances</font>
    * <font size=3>Trading a high bias for a lower variance</font>
    * <font size=3>Speeding up training</font>

In [ ]:
gbrt = GradientBoostingRegressor(max_depth = 2, warm_start = True)

min_val_error = float('inf')
error_going_up = 0
for n_estimators in range(1, 120):
    gbrt.n_estimators =  n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5: # if when the validation error does not improve for five iterations in a row
            break # early stopping


#### **5.3 Stacking (staked generalization)**
* <font size=3>Train a model to perform the aggregation of the predicitons of all predictors</font>
<img src='fig7_12.png' width=600>
* <font size=3>**Training steps** (using a <font color=red>*hold-out set*</font>)</font> 
  * <font size=3>Split the dataset to two: the first subset for traning the predicitons in the first layer</font>
  * <font size=3>The first layer predictors are used to make predictions on the second (held-out) set</font>
  * <font size=3>This generates three predicted values for each instance, forming a new input of three features</font>
  * <font size=3>The blender is trained on the new input to predict the target value given the first layer's predictions
    <img src='fig7_13.png' width =400><img src='fig7_14.png' width =400>
  * <font size=3 color=brown>Multilayer Stacking Ensemble</font>
    * <font size=3>1. Training several different blenders with different algorithms (e.g., Linear Regression, Random Forest Regression)</font>
    * <font size=3>2. Split training set into three subsets</font>
      * <font size=3>2.1 The first one train the first layer,</font>
      * <font size=3>2.2 the second one uses predictions made by the predictors of the first layer</font>
      * <font size=3>2.3 the third one uses predictions made by the predictors of the second layer</font>
      * <font size=3>2.4 Making the prediction for a new instance by going through each layer sequentially</font>
<img src='fig7_15.png' width=500>
    